In [22]:
import json
import logging
import os

import pandas as pd


### Globals

In [23]:
LANG_CODES_TO_NAMES = {
    "en": "English",
    "ru": "Russian",
    "ko": "Korean",
    "cmn-Hant": "Traditional Chinese",
    "ja": "Japanese",
    "de": "German",
    "es": "Spanish",
}
logger = logging.getLogger(__name__)

### Helper functions

In [24]:
# Function to get the script directory
def get_script_dir():
    """Returns the directory where the script is located."""
    dir = os.getcwd()
    if dir.endswith("overrideData"):
        return dir[:-len("overrideData")]
    return dir

# Function to generate file path
def get_file_path(base_dir: str, file: str, lang: str, is_en=False):
    return f"{base_dir}{lang if not is_en else 'en'}/{file}.json"

# Function to load a JSON file
def load_file(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    logger.info(f"LOADING FILE: ++ {path}")
    with open(path, encoding="utf-8") as f:
        return json.load(f)

# Function to load a DataFrame
def load_df(base_dir: str, file: str, lang: str, is_en=False):
    path = get_file_path(base_dir, file, lang, is_en)
    logger.info(f"LOADING FILE: ++ {path}")
    return pd.read_json(path, encoding="utf-8")


### Setting more globals

In [25]:
cwd = get_script_dir()
base_dir = cwd + "/tables/"
out_file = cwd + "/overrideData/logbook_overrides.json"
lang = "en"

Load all tables needed for annoints

## Load json data now

This is correct now, but we need to load in the items from the api side of it now

In [26]:
raw_json_data = {
    group['id']: group
    for group 
    in json.loads(
            open(
                f"{cwd}/../json-api/{lang}/stats.json", encoding="utf-8"
            ).read()
        )['result']
    }

In [27]:
FACTION_STATS_OLD = [{"ref":"Has Logbook Faction: Black Scythe Mercenaries","better":0,"matchers":[{"string":"Black Scythe Mercenaries"}],"trade":{"ids":{"pseudo":["pseudo.pseudo_logbook_faction_mercenaries"]}}},
{"ref":"Has Logbook Faction: Druids of the Broken Circle","better":0,"matchers":[{"string":"Druids of the Broken Circle"}],"trade":{"ids":{"pseudo":["pseudo.pseudo_logbook_faction_druids"]}}},
{"ref":"Has Logbook Faction: Knights of the Sun","better":0,"matchers":[{"string":"Knights of the Sun"}],"trade":{"ids":{"pseudo":["pseudo.pseudo_logbook_faction_knights"]}}},
{"ref":"Has Logbook Faction: Order of the Chalice","better":0,"matchers":[{"string":"Order of the Chalice"}],"trade":{"ids":{"pseudo":["pseudo.pseudo_logbook_faction_order"]}}}]

In [28]:
expedition_factions_en = load_df(base_dir, "ExpeditionFactions", lang).drop(columns=["_index", "Id"])
factions = expedition_factions_en
factions["ref"] = expedition_factions_en["Name"].apply(lambda x: f"Has Logbook Faction: {x}")
factions["better"] = 0
factions["matchers"] = factions["Name"].apply(lambda x: [{"string": x}])
factions["trade"] = None
factions.to_dict(orient="records")



[{'Name': 'Druids of the Broken Circle',
  'ref': 'Has Logbook Faction: Druids of the Broken Circle',
  'better': 0,
  'matchers': [{'string': 'Druids of the Broken Circle'}],
  'trade': None},
 {'Name': 'Black Scythe Mercenaries',
  'ref': 'Has Logbook Faction: Black Scythe Mercenaries',
  'better': 0,
  'matchers': [{'string': 'Black Scythe Mercenaries'}],
  'trade': None},
 {'Name': 'Order of the Chalice',
  'ref': 'Has Logbook Faction: Order of the Chalice',
  'better': 0,
  'matchers': [{'string': 'Order of the Chalice'}],
  'trade': None},
 {'Name': 'Knights of the Sun',
  'ref': 'Has Logbook Faction: Knights of the Sun',
  'better': 0,
  'matchers': [{'string': 'Knights of the Sun'}],
  'trade': None}]

In [29]:
expedition_factions_en

,Name,ref,better,matchers,trade
0,Druids of the Broken Circle,Has Logbook Faction: Druids of the Broken Circle,0,[{'string': 'Druids of the Broken Circle'}],None
1,Black Scythe Mercenaries,Has Logbook Faction: Black Scythe Mercenaries,0,[{'string': 'Black Scythe Mercenaries'}],None
2,Order of the Chalice,Has Logbook Faction: Order of the Chalice,0,[{'string': 'Order of the Chalice'}],None
3,Knights of the Sun,Has Logbook Faction: Knights of the Sun,0,[{'string': 'Knights of the Sun'}],None


In [30]:
output = {}
for language in LANG_CODES_TO_NAMES.keys():
    expedition_factions = load_df(base_dir, "ExpeditionFactions", language).drop(columns=["_index", "Id"])
    expedition_factions["ref"] = expedition_factions_en["Name"].apply(lambda x: f"Has Logbook Faction: {x}")
    expedition_factions["better"] = 0
    expedition_factions["matchers"] = expedition_factions["Name"].apply(lambda x: [{"string": x}])
    expedition_factions["trade"] =expedition_factions_en["Name"].apply(lambda x:  {"ids": None})
    expedition_factions["id"] = expedition_factions_en["Name"].apply(lambda x: x.lower().replace(" ", "_"))
    expedition_factions.drop(columns=["Name"], inplace=True)
    output[language] = expedition_factions.to_dict(orient="records")

In [31]:
with open(
            out_file,
            "w",
            encoding="utf-8",
        ) as f:
            f.write(json.dumps(output, indent=4, ensure_ascii=False))